In [1]:
import pandas as pd
import numpy as np
import microdf as mdf
import scf

s_raw = scf.load(2019, ["networth", "race", "famstruct", "kids", "income"])
s = s_raw.copy(deep=True)
# code races
s["race2"] = np.where(
    s.race.isin([1]),
    "White",  # Not Including Hispanic.
    np.where(s.race == 2, "Black", np.where(s.race == 3, "Hispanic", "Other")),
)

# famstruct 4 and 5 indicate married/LWP (living with partner)
s["numper"] = 1 + s.famstruct.isin([4, 5]) + s.kids
s["adults"] = 1 + s.famstruct.isin([4, 5])

# divide by number of adults
s["networth_pa"] = s.networth / (1 + s.famstruct.isin([4, 5]))

# Calculate tax base by finding weighted sum of individuals and their income.
totals = mdf.weighted_sum(s, ["numper", "income"], w="wgt")
totals.white_hhs = s[s.race2 == "White"].wgt.sum()
totals.black_hhs = s[s.race2 == "Black"].wgt.sum()
totals.total_hhs = s.wgt.sum()


def ubi_sim(data, max_monthly_payment, step_size):
    # Initialize empty list to store our results.
    l = []

    # loop through ubi
    for monthly_payment in np.arange(0, max_monthly_payment + 1, step_size):
        # multiply monthly payment by 12 to get annual payment size
        annual_payment = monthly_payment * 12

        # calculate simulation-level stats
        sim_data = data.copy(deep=True)
        ubi_total = annual_payment * totals.numper
        tax_rate = ubi_total / totals.income
        sim_data["tax_rate"] = tax_rate
        sim_data["ubi_mo"] = monthly_payment
        sim_data["annual_payment"] = annual_payment
        sim_data["networth_new"] = (
            sim_data.networth
            + (annual_payment * sim_data.numper)
            - tax_rate * sim_data.income
        )
        sim_data["networth_pa_new"] = sim_data.networth_new / sim_data.adults
        l.append(sim_data)
    # Return the DataFrames together.
    return pd.concat(l)


# Run the simulation
simulated = ubi_sim(s, 2000, 100)

# create empty list to store candidates for D-statistics
lins = np.array([])
for i in range(4, 6):  # range(2, 9):
    increment = 10 ** (i - 2)
    tmp = np.unique(
        np.round(np.arange(10 ** i, 10 ** (i + 1), increment * 5), -2)
    )
    lins = np.concatenate([lins, tmp])


def shares_below_thresh(data, white_data, black_data, thresh):
    return pd.Series(
        {
            "white_share": white_data[
                white_data.networth_pa_new < thresh
            ].wgt.sum()
            / totals.white_hhs,
            "black_share": black_data[
                black_data.networth_pa_new < thresh
            ].wgt.sum()
            / totals.black_hhs,
            "total_share": data[data.networth_pa_new < thresh].wgt.sum()
            / totals.total_hhs,
        }
    )


cdf_list = []
for step in simulated.ubi_mo.unique():
    data = simulated[simulated.ubi_mo == step]
    white_data = data[data.race2 == "White"]
    black_data = data[data.race2 == "Black"]
    # create df generating networths evenly spaced when we view in log form
    cdf = pd.DataFrame({"networth_pa_new": lins, "ubi_mo": step})
    cdf = pd.concat(
        [
            cdf,
            cdf.networth_pa_new.apply(
                lambda x: shares_below_thresh(data, white_data, black_data, x)
            ),
        ],
        axis=1,
    )
    cdf["d_stat_cand"] = cdf.black_share - cdf.white_share
    cdf_list.append(cdf)

cdfs = pd.concat(cdf_list)

# Create DataFrame summarized at the UBI amount, with columns for:
# - d_stat and associated net worth
# - median and mean net worth by white/black
# - share with net worth above $50k by white/black
cdfs_max = (
    cdfs.sort_values("d_stat_cand", ascending=False).groupby("ubi_mo").head(1)
)
ubi_summary = simulated.groupby("ubi_mo").apply(
    lambda x: pd.Series(
        {
            "black_median_networth_pa": mdf.weighted_median(
                x[x.race2 == "Black"], "networth_pa", "wgt"
            ),
            "white_median_networth_pa": mdf.weighted_median(
                x[x.race2 == "White"], "networth_pa", "wgt"
            ),
            "black_mean_networth_pa": mdf.weighted_median(
                x[x.race2 == "Black"], "networth_pa", "wgt"
            ),
            "white_mean_networth_pa": mdf.weighted_median(
                x[x.race2 == "White"], "networth_pa", "wgt"
            ),
            "black_share_above_50k": x[
                (x.race2 == "Black") & (x.networth_pa >= 50000)
            ].wgt.sum()
            / totals.black_hhs,
            "white_share_above_50k": x[
                (x.race2 == "White") & (x.networth_pa >= 50000)
            ].wgt.sum()
            / totals.white_hhs,
        }
    )
).reset_index()
ubi_summary["white_mean_nw_as_pct_of_mean_black"] = (
    ubi_summary.white_mean_networth_pa / ubi_summary.black_mean_networth_pa
)
ubi_summary["white_median_nw_as_pct_of_median_black"] = (
    ubi_summary.white_median_networth_pa / ubi_summary.black_median_networth_pa
)
ubi_summary["white_share_above_50k_pct_of__black"] = (
    ubi_summary.white_share_above_50k / ubi_summary.black_share_above_50k
)

ubi_summary = ubi_summary.merge(cdfs_max, on="ubi_mo").reset_index()

cdfs.to_csv("cdfs.csv", index=False)
ubi_summary.to_csv("cdfs_max.csv", index=False)

In [6]:
ubi_summary = simulated.groupby("ubi_mo").apply(
    lambda x: pd.Series(
        {
            "black_median_networth_pa": mdf.weighted_median(
                x[x.race2 == "Black"], "networth_pa_new", "wgt"
            ),
            "white_median_networth_pa": mdf.weighted_median(
                x[x.race2 == "White"], "networth_pa_new", "wgt"
            ),
            "black_mean_networth_pa": mdf.weighted_median(
                x[x.race2 == "Black"], "networth_pa_new", "wgt"
            ),
            "white_mean_networth_pa": mdf.weighted_median(
                x[x.race2 == "White"], "networth_pa_new", "wgt"
            ),
            "black_share_above_50k": x[
                (x.race2 == "Black") & (x.networth_pa_new >= 50000)
            ].wgt.sum()
            / totals.black_hhs,
            "white_share_above_50k": x[
                (x.race2 == "White") & (x.networth_pa_new >= 50000)
            ].wgt.sum()
            / totals.white_hhs,
        }
    )
)
ubi_summary["white_mean_nw_as_pct_of_mean_black"] = (
    ubi_summary.white_mean_networth_pa / ubi_summary.black_mean_networth_pa
)
ubi_summary["white_median_nw_as_pct_of_median_black"] = (
    ubi_summary.white_median_networth_pa / ubi_summary.black_median_networth_pa
)
ubi_summary["white_share_above_50k_pct_of__black"] = (
    ubi_summary.white_share_above_50k / ubi_summary.black_share_above_50k
)
ubi_summary

,black_median_networth_pa,white_median_networth_pa,black_mean_networth_pa,white_mean_networth_pa,black_share_above_50k,white_share_above_50k,white_mean_nw_as_pct_of_mean_black,white_median_nw_as_pct_of_median_black,white_share_above_50k_pct_of__black
ubi_mo,,,,,,,,,
0,17853.413512,115103.205176,17853.413512,115103.205176,0.341684,0.666678,6.447126,6.447126,1.951153
100,19687.422118,115047.597915,19687.422118,115047.597915,0.347011,0.668764,5.843711,5.843711,1.927211
200,20304.336241,115819.620500,20304.336241,115819.620500,0.350840,0.670469,5.704182,5.704182,1.911040
300,21531.403356,116238.317638,21531.403356,116238.317638,0.353395,0.671046,5.398548,5.398548,1.898856
400,23139.572839,116555.429515,23139.572839,116555.429515,0.357689,0.672625,5.037061,5.037061,1.880475
500,24981.357230,117488.462746,24981.357230,117488.462746,0.361546,0.674673,4.703046,4.703046,1.866077
600,25935.937027,117278.977960,25935.937027,117278.977960,0.363914,0.676957,4.521872,4.521872,1.860213
700,28070.683575,117698.711471,28070.683575,117698.711471,0.369726,0.677720,4.192941,4.192941,1.833032
800,29595.863493,118704.677990,29595.863493,118704.677990,0.377953,0.679425,4.010854,4.010854,1.797642


In [5]:
simulated

,famstruct,wgt,income,kids,networth,race,race2,numper,adults,networth_pa,tax_rate,ubi_mo,annual_payment,networth_new,networth_pa_new
0,3,6119.779308,67195.781504,0,2153600.0,1,White,1,1,2153600.0,0.000000,0,0,2.153600e+06,2.153600e+06
1,3,4712.374912,57014.602488,0,2116200.0,1,White,1,1,2116200.0,0.000000,0,0,2.116200e+06,2.116200e+06
2,3,5145.224455,51924.012980,0,2145000.0,1,White,1,1,2145000.0,0.000000,0,0,2.145000e+06,2.145000e+06
3,3,5297.663412,41742.833964,0,2552500.0,1,White,1,1,2552500.0,0.000000,0,0,2.552500e+06,2.552500e+06
4,3,4761.812371,50905.895078,0,2176200.0,1,White,1,1,2176200.0,0.000000,0,0,2.176200e+06,2.176200e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28880,5,667.098300,411319.632234,0,7635200.0,1,White,2,2,3817600.0,0.515647,2000,24000,7.471104e+06,3.735552e+06
28881,5,678.821856,410301.514332,0,7728800.0,1,White,2,2,3864400.0,0.515647,2000,24000,7.565229e+06,3.782615e+06
28882,5,640.908142,379757.977285,0,7485800.0,1,White,2,2,3742900.0,0.515647,2000,24000,7.337979e+06,3.668990e+06
28883,5,665.152072,430663.872363,0,7995500.0,1,White,2,2,3997750.0,0.515647,2000,24000,7.821430e+06,3.910715e+06


In [2]:

# Install microdf
!pip install git+https://github.com/PSLmodels/microdf.git
# update plotly
!pip install plotly --upgrade

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import microdf as mdf
import warnings
import plotly.io as pio
# pio.templates

# Turn off display bar
CONFIG = {"displayModeBar": False}

# Define UBI Center colors
BLUE = "#1976D2"
DARK_BLUE = "#1565C0"
LIGHT_BLUE = "#90CAF9"
GRAY = "#BDBDBD"
BARELY_BLUE = "#E3F2FD"

colors=[BLUE, DARK_BLUE, LIGHT_BLUE, GRAY, BARELY_BLUE]

  Cloning https://github.com/PSLmodels/microdf.git to /tmp/pip-req-build-ktyih2fp
  Running command git clone -q https://github.com/PSLmodels/microdf.git /tmp/pip-req-build-ktyih2fp
  Created wheel for microdf: filename=microdf-0.2.0-cp37-none-any.whl size=26117 sha256=ca40a24fcd8790d26b8d096487238a5ce1ee3ab3cf18ba53bbfc16d0cc4b569e
  Stored in directory: /tmp/pip-ephem-wheel-cache-6mkstucc/wheels/3d/53/af/92e56f83db191b0579d21e8385d61a92a502e66443b23c7e16
Successfully built microdf
     |████████████████████████████████| 13.2MB 328kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [3]:
# install scf
!pip install git+https://github.com/PSLmodels/scf.git
import scf

  Cloning https://github.com/PSLmodels/scf.git to /tmp/pip-req-build-tnhc7ngx
  Running command git clone -q https://github.com/PSLmodels/scf.git /tmp/pip-req-build-tnhc7ngx
  Created wheel for scf: filename=scf-0.1.0-cp37-none-any.whl size=2402 sha256=219cdb537052db83ba52ec9f38441d9e3e0d2de4acd2e8a4c0982cc55157d0b1
  Stored in directory: /tmp/pip-ephem-wheel-cache-gts6tswj/wheels/d0/48/e8/e1e1a7e4acf46ac2db392b983862af50f1630fadffb27bd9f3
Successfully built scf


The large racial net worth gap traces its roots to slavery, redlining, and other discriminatory policies, and persists largely due to [racial income gaps](https://www.bloomberg.com/news/articles/2019-03-21/how-income-inequality-feeds-the-racial-wealth-gap). In honor of Black History month, we explore how closing part of this income gap with a universal basic income would affect the racial net worth gap, using novel measurements that consider how Black and White families differ across the net worth distribution.

The two most common methods to compare distributions is to compare the mean and median household net worth held by Black families and that held by White families. 


In [4]:
s_raw = scf.load(2019,['networth', 'race', 'famstruct', 'kids','income'])
s = s_raw.copy(deep=True)
# code races
s["race2"] = np.where(
    s.race.isin([1]),
    "White",  # Not Including Hispanic.
    np.where(s.race == 2, "Black", np.where(s.race == 3, "Hispanic", "Other")),
)

# famstruct 4 and 5 indicate married/LWP (living with partner)
s["numper"] = 1 + s.famstruct.isin([4, 5]) + s.kids
s["adults"] = 1 + s.famstruct.isin([4, 5])

# divide by number of adults
s["networth_pa"] = s.networth / (1 + s.famstruct.isin([4, 5]))

def add_ubi_center_logo(fig, x=0.98, y=-0.14):
  '''returns UBI Center logo on plotly graphs'''
  fig.add_layout_image(
      dict(
          source="https://raw.githubusercontent.com/UBICenter/blog/master/jb/_static/ubi_center_logo_wide_blue.png",
          # See https://github.com/plotly/plotly.py/issues/2975.
          # source="../_static/ubi_center_logo_wide_blue.png",
          xref="paper", yref="paper",
          x=x, y=y,
          sizex=0.12, sizey=0.12,
          xanchor="right", yanchor="bottom"
      )
  )

def common_layout_changes(fig):
  fig.update_layout(
      font=dict(family="Roboto"),
      template=template,
      width=1000,
      height=600
      )
  

def format_helper(fig, 
    title,
    xaxis_title,
    yaxis_title,
    yaxis_ticksuffix="",
    yaxis_tickprefix="", 
    xaxis_tickprefix="",
    hovermode="x"):
  
  fig.update_layout(
      title=title,
      xaxis_title=xaxis_title,
      yaxis_title=yaxis_title,
      yaxis_ticksuffix=yaxis_ticksuffix,
      yaxis_tickprefix=yaxis_tickprefix,
      hovermode=hovermode,
      xaxis_tickprefix=xaxis_tickprefix,
      plot_bgcolor="white",
      legend_title_text="",
      font=dict(family="Roboto"),
      width=1000,
      height=600,
      )

  

  # add ubi center logo
  fig.add_layout_image(
        dict(
            source="https://raw.githubusercontent.com/UBICenter/blog/master/jb/_static/ubi_center_logo_wide_blue.png",
            # See https://github.com/plotly/plotly.py/issues/2975.
            # source="../_static/ubi_center_logo_wide_blue.png",
            xref="paper", yref="paper",
             x=0.98, y=-0.14,
            sizex=0.12, sizey=0.12,
            xanchor="right", yanchor="bottom"
        ))
  
# set template
template='simple_white'

# calculate weighted mean and median grouped by race
params = s.groupby("race2").apply(
    lambda x: pd.Series(
        {
            "mean": mdf.weighted_mean(x, "networth_pa", "wgt"),
            "median": mdf.weighted_median(x, "networth_pa", "wgt"),
        }
    )
)

# transpose the parameters dataframe
params_T = params.T

def grouped_bar(df):
  # create figure with all weighted median and mean metrics for each race
  fig = go.Figure(
      data=[
          go.Bar(name="Black", x=df.index, y=df["Black"],marker_color=BLUE),
          go.Bar(name="White", x=df.index, y=df["White"],marker_color=GRAY),
      ]
  )
  return fig

fig=grouped_bar(params.T)

format_helper(
    fig,
    title="Racial Wealth Gap - 2019",
    hovermode="x",
    xaxis_title="",
    yaxis_title="Net Worth per Adult",   
    yaxis_tickprefix="$"
)

fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.75))

fig.show(config=CONFIG)

As Matt Bruenig pointed out in his [June 2020 piece](https://www.peoplespolicyproject.org/2020/06/29/the-racial-wealth-gap-is-about-the-upper-classes/), the mean net worth gap is largely shaped by overall net worth inequality, with the vast majority of net worth held by both Black and White households being held by the top decile of each. 

In [5]:
# kolmogorov-smirnoff
def ks_w2(data1, data2, wei1, wei2):
    """
    args:
      data1:  first pandas series or numpy array
      data2:  second pandas series or numpy array
      wei1: pd series or numpy array containing weights
      applied to values in data1
      wei2: wei1: pd series or numpy array containing weights
      applied to values in data2

    returns:
      kolmogorov-smirnoff test statistic for two weighted series
    """
    data1 = np.array(tuple(data1))
    data2 = np.array(tuple(data2))
    wei1 = np.array(tuple(wei1))
    wei2 = np.array(tuple(wei2))
    ix1 = np.argsort(data1)
    ix2 = np.argsort(data2)
    data1 = data1[ix1]
    data2 = data2[ix2]
    wei1 = wei1[ix1]
    wei2 = wei2[ix2]
    data = np.concatenate([data1, data2])
    cwei1 = np.hstack([0, np.cumsum(wei1) / sum(wei1)])
    cwei2 = np.hstack([0, np.cumsum(wei2) / sum(wei2)])
    cdf1we = cwei1[[np.searchsorted(data1, data, side="right")]]
    cdf2we = cwei2[[np.searchsorted(data2, data, side="right")]]
    return np.max(np.abs(cdf1we - cdf2we))


def ecdf(df, value, weight):
    """Compute ECDF for a one-dimensional array of measurements."""

    # Number of data points: n
    n = df[weight].sum()

    # x-data for the ECDF: x
    df.sort_values(value, inplace=True)
    x = df[value]
    y = df[weight].cumsum() / n

    # cumulative sum of the weights
    return x, y

def ubi_sim(data, max_monthly_payment, step_size):

    # initialize empty dataframe to store our results
    big_df = pd.DataFrame()

    # calculate tax base by finding weighted some of individuals and their income
    totals = mdf.weighted_sum(data, ["numper", "income"], w="wgt")

    # loop through ubi
    for monthly_payment in np.arange(0, max_monthly_payment + 1, step_size):
        # multiply monthly payment by 12 to get annual payment size
        annual_payment = monthly_payment * 12

        # calculate simulation-level stats
        sim_data = data.copy(deep=True)
        ubi_total = annual_payment * totals.numper
        tax_rate = ubi_total / totals.income
        sim_data["tax_rate"] = tax_rate
        sim_data["ubi_mo"] = monthly_payment
        sim_data["annual_payment"] = annual_payment
        sim_data["networth_new"] = (
            sim_data.networth
            + (annual_payment * sim_data.numper)
            - tax_rate * sim_data.income
        )
        sim_data["networth_pa_new"] = sim_data.networth_new / sim_data.adults

        # loop through each race to calculate ecdf columns
        for race2 in data.race2.unique():
            race_df = sim_data[sim_data["race2"] == race2].copy(deep=True)
            race_df["ecdf_x"], race_df["ecdf_y"] = ecdf(
                race_df, "networth_pa_new", "wgt"
            )
            # append data to big dataframe
            big_df = big_df.append(race_df)
    big_df.index.name = "scf_id"
    # index by ubi size and scf index id
    return big_df

# run the simulation
simulated = ubi_sim(s, 2000, 100)

In [6]:
# create empty list to store candidates for D-statistics
lins = np.array([])
for i in range(2, 9):
    if i < 3:
        increment = 10 ** (i - 2)
    else:
        increment = 10 ** (i - 2)
    tmp = np.unique(np.round(np.arange(10 ** i, 10 ** (i + 1), increment*5), -2))
    lins = np.concatenate([lins, tmp])


    
print("linspace length: " +str(len(lins)))
df = pd.DataFrame(
    {"x": range(0, len(lins)), "networth_pa": lins, "networth_pa_log": np.log(lins)}
)

px.scatter(df, "x", "networth_pa_log", hover_data=["networth_pa"])

linspace length: 1001


In [ ]:
# create empty list to store candidates for D-statistics
lins = np.array([])
for i in range(2, 9):
    if i < 3:
        increment = 10 ** (i - 2)
    else:
        increment = 10 ** (i - 2)
    tmp = np.unique(np.round(np.arange(10 ** i, 10 ** (i + 1), increment*5), -2))
    lins = np.concatenate([lins, tmp])


cdf_list=[]
for step in simulated.ubi_mo.unique():
  data = simulated[simulated.ubi_mo==step]
  # create df generating networths evenly spaced when we view in log form
  cdf = pd.DataFrame({
      "networth_pa_new": lins,
      'ubi_mo':step
      })
  cdf["white_share"] = cdf.networth_pa_new.apply(
      lambda x: data[(data.race2 == "White") & (data.networth_pa_new < x)].wgt.sum()
      / s[(s.race2 == "White")].wgt.sum()
  )
  cdf["black_share"] = cdf.networth_pa_new.apply(
      lambda x: data[(data.race2 == "Black") & (data.networth_pa_new < x)].wgt.sum()
      / s[(s.race2 == "Black")].wgt.sum()
  )
  cdf["d_stat_cand"] = cdf.black_share - cdf.white_share
  cdf_list.append(cdf)

cdfs = pd.concat(cdf_list).reset_index()

cdfs_max = cdfs.sort_values('d_stat_cand', ascending=False).groupby('ubi_mo').head(1)

In [ ]:
cdfs.sample(25)

In [ ]:


# find the index of the closet value in column to a specified value
def find_neighbours(value, df, colname):
    exactmatch = df[df[colname] == value]
    if not exactmatch.empty:
        return exactmatch.index
    else:
        lowerneighbour_ind = df[df[colname] < value][colname].idxmax()
        upperneighbour_ind = df[df[colname] > value][colname].idxmin()
        return [lowerneighbour_ind, upperneighbour_ind] 

In [ ]:
def sim_stats(data, race_a, race_b):
    list_of_dicts = []
    for i, ubi in enumerate(data.ubi_mo.unique()):
        a = data[(data.ubi_mo == ubi) & (data.race2 == race_a)].copy(deep=True)
        b = data[(data.ubi_mo == ubi) & (data.race2 == race_b)].copy(deep=True)
        # calculate means & medians for both races
        round_to = -2
        a_mean, a_median = (
            mdf.weighted_mean(a, "networth_pa_new", "wgt"),
            mdf.weighted_median(a, "networth_pa_new", "wgt"),
        )
        b_mean, b_median = (
            mdf.weighted_mean(b, "networth_pa_new", "wgt"),
            mdf.weighted_median(b, "networth_pa_new", "wgt"),
        )
        # calculate KS d-statistic
        ks_stat = ks_w2(a.networth_pa_new, b.networth_pa_new, a.wgt, b.wgt)

        
        # find networth where the d-statistic
        max_x = cdfs_max.loc[cdfs_max.ubi_mo==ubi, 'networth_pa_new'].max()

        # find networth where abs distance between ecdfs is is @  max
        a_ks_y = cdfs_max.loc[cdfs_max.ubi_mo==ubi, 'white_share'].max()
        b_ks_y = cdfs_max.loc[cdfs_max.ubi_mo==ubi, 'black_share'].max()
        # find share of households with nw less than $50k
        # a_above_50k = 1-a.loc[find_neighbours(50000, a, "ecdf_x")[0], "ecdf_y"]
        a_above_50k = 1-cdfs.loc[(cdfs.ubi_mo==ubi)&(cdfs.networth_pa_new==50000),'white_share'].max()
        # b_above_50k = 1-b.loc[find_neighbours(50000, b, "ecdf_x")[0], "ecdf_y"]
        b_above_50k= 1-cdfs.loc[(cdfs.ubi_mo==ubi)&(cdfs.networth_pa_new==50000),'black_share'].max()
        # define dictionary of all these values to append to list of dicts
        d = {
            "ubi_mo": ubi,
            race_a + "_mean_networth_pa": a_mean,
            race_a + "_median_networth_pa": a_median,
            race_b + "_mean_networth_pa": b_mean,
            race_b + "_median_networth_pa": b_median,
            race_a + "_mean_nw_as_pct_of_mean_" + race_b: a_mean / b_mean*100,
            race_a + "_median_nw_as_pct_of_median_" + race_b: a_median / b_median*100,
            "ks_stat": ks_stat,
            "most_unequal_networth_pa": max_x,
            race_a + "_share_above_50k": a_above_50k,
            race_b + "_share_above_50k": b_above_50k,
            race_a + "_share_above_50k_pct_of_" + race_b: a_above_50k / b_above_50k*100,
            race_a + "_ecdf_at_ks": a_ks_y,
            race_b + "_ecdf_at_ks": b_ks_y,
             'tax_rate':data.tax_rate.max()
        }
        list_of_dicts.append(d)
    return pd.DataFrame(list_of_dicts)


with warnings.catch_warnings():  # ignore "FutureWarning"
    """
    Ignore numpy 'FutureWarning' :
    "Using a non-tuple sequence for multidimensional indexing is deprecated;
    use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be
    interpreted as an array index, `arr[np.array(seq)]`, which will result
    either in an error or a different result."
    """
    warnings.filterwarnings("ignore", category=FutureWarning)
    sim_results = sim_stats(simulated, "White","Black")

# create list of four selected measurements of racial wealth gap
cols = [
    "White_mean_nw_as_pct_of_mean_Black",
    "White_median_nw_as_pct_of_median_Black",
    "ks_stat",
    "White_share_above_50k_pct_of_Black",
]

# create 4 columns measuring the relative change of 4 measurements from baseline
metric_cols = []
for col in cols:
    # define string with new name - '_deltap' for change in percent
    name = col + "_deltap"
    metric_cols.append(name)
    sim_results[name] = (
        (sim_results[col] - sim_results[col][0]) / sim_results[col][0] * 100
    )



# Plot quintile by race

In [ ]:
hovertemplate="%{x:.1f}%"

def total_weath_by_decile(data, measure):
    quant_df = pd.DataFrame()
    for race2 in data.race2.unique():
        race_df = data[data.race2 == race2].copy(deep=True)
        decile_bounds = np.arange(0, 1.1, 0.1)
        deciles = mdf.weighted_quantile(race_df, measure, "wgt", decile_bounds)

        race_total_nw = mdf.weighted_sum(race_df, measure, "wgt")
        quantile_nws = []
        for index, value in enumerate(deciles):
            if index + 1 < len(deciles):
                quantile_subset = race_df[
                    race_df.networth.between(value, deciles[index + 1])
                ]
                quantile_nws.append(mdf.weighted_sum(quantile_subset, measure, "wgt"))
        quantile_nw_pct = (quantile_nws / race_total_nw) * 100
        race_quant_df = pd.DataFrame(
            {race2: quantile_nw_pct}, index=np.arange(1, 11, 1)
        )
        quant_df = pd.concat([quant_df, race_quant_df], axis=1)
    return quant_df


nw_quant = (
    total_weath_by_decile(s, "networth")
    .reset_index()
    .rename(columns={"index": "percentile"})
    .melt(id_vars=["percentile"])
)

fig = px.bar(
    nw_quant[nw_quant.variable.isin(['White',"Black"])],
    y="percentile",
    x="value",
    color="variable",
    labels={
        "percentile": "Percent receiving benefit",
        "value": "Share of Within Group Wealth",
    },
    orientation="h",
    barmode="group",
    color_discrete_map={"White": GRAY, "Black": BLUE},
)


fig.update_traces(hovertemplate="Share of Wealth: "+hovertemplate)


# apply formatting
format_helper(
    fig,
    title="Household Inequality by Race",
    hovermode="y",
    xaxis_title="Share of total within-group wealth held by decile",
    yaxis_title="Wealth decile",   
    yaxis_ticksuffix="%"
)


fig.show(config=CONFIG)


While the median tells us where the middle of the distributions lay, it can obscure what is going on throughout the rest of the distribution. 

One tool that we can use to visualize a variable’s distribution in full is with a cumulative distribution function. A cumulative distribution function (CDF) tells us the probability a random variable returns a value less than some specified value. 

To show how to interpret the graph of the CDF, see the following graph showing the distribution of household net worth per adult for all households in the U.S. (Note that while we have logarithmically transformed the x-axis, the y-axis remains linear.) 


In [ ]:



df1 = s.copy(deep=True)
df1["x"], df1["y"] = ecdf(df1, "networth_pa", "wgt")
df1["weighted_networth_pa"] = df1.networth_pa * df1.wgt
# select median. This isn't the weighted median, but it will do for illustrative purposes
# since we're referring to any given pt on x
med = df1.loc[df1["weighted_networth_pa"] == df1["weighted_networth_pa"].median()].max()
med.x = np.log10(med.x)

# plot curve
fig = px.line(x=df1.x, y=df1.y, template=template)
# change line coloar
fig["data"][0]["line"]["color"] = BLUE

# add explanation for arrow pointing to y-axis
fig.add_annotation(
    x=0,
    y=med.y,
    text="Look at y-axis to see share of households <br> whose net worth is below the x-value",
    showarrow=True,
    arrowhead=0,
    ay=-50,
    ax=150,
)

# add dotted line pointing from x-axis to point
fig.add_annotation(
    x=med.x,
    y=0,
    text="For a given net worth on the x-axis",
    showarrow=True,
    arrowhead=0,
    # yshift=-60,
    # xshift=-130,
    ax=160,
    ay=-50,
)

# add red arrow pointing from x-axis to point
fig.add_annotation(
    x=med.x,  # arrows' head
    y=med.y,  # arrows' head
    ax=med.x,  # arrows' tail
    ay=0,  # arrows' tail
    xref="x",
    yref="y",
    axref="x",
    ayref="y",
    text="",  # if you want only the arrow
    showarrow=True,
    arrowhead=2,
    arrowsize=2,
    arrowwidth=1,
    arrowcolor=GRAY,
)

# add arrow pointing from point to y-axis
fig.add_annotation(
    x=0,  # arrows' head
    y=med.y,  # arrows' head
    ax=med.x,  # arrows' tail
    ay=med.y,  # arrows' tail
    xref="x",
    yref="y",
    axref="x",
    ayref="y",
    text="",  # if you want only the arrow
    showarrow=True,
    arrowhead=2,
    arrowsize=2,
    arrowwidth=1,
    arrowcolor=GRAY,
)

fig.update_layout(showlegend=False)

format_helper(
    fig,
    title="CumulativeDistribution Function of Household Net Worth per Adult in 2019",
    xaxis_title="Net worth (USD)",
    yaxis_title="CDF",
    yaxis_ticksuffix="",
    yaxis_tickprefix="",
    xaxis_tickprefix="$",
    hovermode="x",
)

fig.update_xaxes(
    dict(
        range=[0, 9],
        autorange=False,
        showspikes=False,  # Show spike line for x-axis
        # Format spike
        spikethickness=1,
        spikedash="dot",
        spikecolor="grey",
        spikemode="toaxis",
    ),
    type="log",
)
fig.update_yaxes(
    dict(
        range=[0, 1],
        showspikes=True,
        spikethickness=1,
        spikedash="dot",
        spikecolor="grey",
        spikemode="toaxis",
    )
)


fig.show()


Now, how can we compare the distributions of net worth between Black and White households? The two-sample Kolmogorov-Smirnov (KS) statistical test offers a way to test the difference between two distributions. The KS statistic, or D-statistic, is the maximum absolute vertical distance between two CDFs. One advantage to this statistical method is that it is quite simple to explain visually. The below chart plots the CDF for White and Black households. The dotted line drawn between the two curves shows where the vertical distance between them is at its greatest.

In [ ]:
with warnings.catch_warnings():  # ignore "FutureWarning"
    """
    Ignore numpy 'FutureWarning' :
    "Using a non-tuple sequence for multidimensional indexing is deprecated;
    use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be
    interpreted as an array index, `arr[np.array(seq)]`, which will result
    either in an error or a different result."
    """
    warnings.filterwarnings("ignore", category=FutureWarning)


    # Create traces
    fig = go.Figure()

    race_cats=[
              'White',
              'Black',
              'Hispanic',
              'Other'
    ]

    # create seperate df for white and black households
    black = s[s.race2 == "Black"].copy(deep=True)
    white = s[s.race2 == "White"].copy(deep=True)
    # calculate ECDFs, add as columns
    black['ecdf_x'], black['ecdf_y'] = ecdf(black,"networth_pa","wgt")
    white['ecdf_x'], white['ecdf_y'] = ecdf(white,"networth_pa","wgt")
    # plot ECDFs
    fig.add_trace(go.Scatter(x=white.ecdf_x, y=white.ecdf_y, name="White",marker_color=GRAY))
    fig.add_trace(go.Scatter(x=black.ecdf_x, y=black.ecdf_y, name="Black",marker_color=BLUE))

    # plot hidden ECDFs for other households
    for race2 in ['Hispanic','Other']:
      X, Y = ecdf(s[s.race2==race2].copy(deep=True),"networth_pa","wgt")
      fig.add_trace(go.Scatter(x=X, y=Y,name=race2,line={'width': .5},
                              visible='legendonly'
                              ))

    ks_stat = ks_w2(
                white.networth_pa, black.networth_pa, white.wgt, black.wgt
            )


    # find networth where the d-statistic
    max_x = cdfs_max.loc[cdfs_max.ubi_mo==0, 'networth_pa_new'].max()
    

    # find networth where abs distance between ecdfs is is @  max
    y1 = cdfs_max.loc[cdfs_max.ubi_mo==0, 'white_share'].max()
    y0 = cdfs_max.loc[cdfs_max.ubi_mo==0, 'black_share'].max()
    # find networth where the d-statistic
    max_x = candf.loc[candf.y == candf.y.max(), "x"].max()


    # # add a dashed line showing where max abs distance between ECDFs is

    y1 = white.loc[find_neighbours(max_x, white, "ecdf_x")[0], "ecdf_y"]
    y0 = black.loc[find_neighbours(max_x, black, "ecdf_x")[0], "ecdf_y"]
    dash_names = "Max abs. distance between ECDFs"
    dash_color= 'black'
    fig.add_trace(
        go.Scatter(
            x=[max_x, max_x],
            y=[y0, y1],
            mode="lines+text",
            line={"dash": "dot", "color": dash_color,'width':2},
            visible=True,
            name=dash_names,
            text=["D-statistic: "+str(round(ks_stat,3))+"        ",None],
            textposition="top left",
            showlegend=False
        ))

    fig.add_annotation(
        x=np.log10(max_x), 
        y=(y1+y0)/2,
                text="The Kolmogorov-Smirnov D-statistic <br>"+\
                "is the maximum absolute vertical <br>"+\
                "distance between the two curves",
                showarrow=True,
                arrowhead=3,
                yshift=1,
                ax=200,
                bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor="dark blue",
            opacity=1
                )



    format_helper(fig, 
      title='Cumulative distribution function (CDF) of wealth by race',
      xaxis_title="Net worth (USD)",
      yaxis_title="CDF",
      yaxis_ticksuffix="",
      yaxis_tickprefix="", 
      xaxis_tickprefix="$",
      )

    fig.update_xaxes(type="log")
    fig.show()

This shows us that approximately 65.9% of Black households have a net worth of less than $50,000 per adult, whereas only 33.3% of White households fall below that line. As we will see, the threshold can be at any point along these curves; the KS statistic is the value measuring the size of the gap where the gap is the largest, at 0.326 (the actual net worth value corresponding to the D-statistic was $50,888).

In the following model, we simulate how saving a total of one year’s UBI payments could change the racial net worth gap. We apply a flat tax on all income to fund the UBI program, and subtract each household’s new tax payment from their net worth. 

In the bottom graph, you can see the vertical distance between the two curves at each point along the x-axis from $0 to $2,000 per month.


In [ ]:
# %%timeit


def ecdf_slider(data):
    # Create traces
    fig = go.Figure()

    # create empty list for payment size for sliders
    payments = []
    ks_stats = []
    max_xs = []

    # create 2 subplots
    fig = make_subplots(rows=2, cols=1, shared_xaxes=False)

    for i, step in enumerate(data.ubi_mo.unique()):
        # append payment to list
        payments.append(step)
        white = data[(data.ubi_mo == step) & (data.race2 == "White")]
        black = data[(data.ubi_mo == step) & (data.race2 == "Black")]
        # add trace 0
        fig.add_trace(
            go.Scatter(
                x=white.ecdf_x,
                y=white.ecdf_y,
                name="White",
                visible=False,
                marker_color=GRAY,
            ),
            row=1,
            col=1,
        )
        # add trace 1
        fig.add_trace(
            go.Scatter(
                x=black.ecdf_x,
                y=black.ecdf_y,
                name="Black",
                visible=False,
                marker_color=BLUE,
            ),
            row=1,
            col=1,
        )
        # append ks-statistics to list
        ks_stat = sim_results.loc[sim_results.ubi_mo==step,'ks_stat'].values[0]
        ks_stats.append(ks_stat)

        # add plot to candidate
        candf = cdfs[cdfs.ubi_mo==step]
        # add trace 2, the d-statistic candidates
        fig.add_trace(
            go.Scatter(
                x=candf.networth_pa_new,
                y=candf.d_stat_cand,
                mode="lines",
                visible=False,
                name="Absolute Distance Between ECDFs",
                marker_color=LIGHT_BLUE
            ),
            row=2,
            col=1,
        )

        # find networth where the d-statistic
        # find networth where the d-statistic
        max_x = cdfs_max.loc[cdfs_max.ubi_mo==step, 'networth_pa_new'].iloc[0]
        max_xs.append(max_x)

        # find networth where abs distance between ecdfs is is @  max
        y1 = cdfs_max.loc[cdfs_max.ubi_mo==step, 'white_share'].iloc[0]
        y0 = cdfs_max.loc[cdfs_max.ubi_mo==step, 'black_share'].iloc[0]

        dash_names = "Max abs. distance between ECDFs"
        dash_color = "black"
        fig.add_trace(
            go.Scatter(
                x=[max_x, max_x],
                y=[y0, y1],
                mode="lines+text",
                line={"dash": "dot", "color": dash_color, "width": 2},
                visible=False,
                name=dash_names,
                text=["D-statistic: " + str(round(ks_stat, 3)) + "     ", None],
                textposition="top left",
                showlegend=False,
            ),
            row=1,
            col=1,
        )
        fig.add_trace(
            go.Scatter(
                x=[max_x, max_x],
                y=[0, candf.d_stat_cand.max()],
                mode="lines",
                line={"dash": "dot", "color": dash_color, "width": 2},
                visible=False,
                name=dash_names,
                showlegend=False,
            ),
            row=2,
            col=1,
        )

        fig.add_annotation(
            x=(max_x),
            y=0,
            # text option 1 for general descriptor
            # text="For a given value on the x-axis",
            # text option for net worth descriptor
            text="For a given net worth on the x-axis",
            showarrow=True,
            arrowhead=0,
            # yshift=-60,
            # xshift=-130,
            # ax=160,
            ay=-50,
        )
    # define number of plots in each step
    n = 5
    # Make first n traces visible
    for i in range(0, n):
        fig.data[i].visible = True

    # Create and add slider
    steps = []
    for i in range(0, len(fig.data), n):
        step = dict(
            method="update",
            args=[
                {"visible": [False] * len(fig.data)},
                {
                    "title": "Kolmogorov-Smirnoff D-statistic: "
                    + str(round(ks_stats[i // n], 3))
                },
                # {'annotations[0].text' : 'External %d' % (i // n)},
                {
                    "annotations": dict(
                        x=max_xs[i // n],
                        y=0,
                        text="External %d" % (i // n),
                    )
                },
                # {'shapes':{'visible':[False] * len(fig.layout.shapes)}}
            ],  # layout attribute
            label=("$" + str(payments[i // n])),
        )
        for j in range(0, n):
            step["args"][0]["visible"][i + j] = True  # Toggle i'th trace to "visible"
        steps.append(step)
    sliders = [
        dict(
            active=0,
            currentvalue={"prefix": "Monthly Benefit: "},
            pad={"t": 50},
            steps=steps,
        )
    ]

    fig.update_traces(xaxis="x1")

    # update shapes
    fig.update_shapes(dict(xref="x", yref="y"))

    # fig.update_yaxes(showspikes=True)
    fig.update_layout(
        sliders=sliders,
        xaxis=dict(
            visible=True,
            type="log",
            range=[0, 9],
            autorange=False,
            showspikes=True,  # Show spike line for X-axis
            # Format spike
            spikethickness=2,
            spikedash="dot",
            spikecolor="#999999",
            spikemode="across",
        ),
        title="Empirical Cumlative Density Function (ECDF) of Wealth by Race",
        xaxis_title="Net Worth (USD)",
        yaxis_title="ECDF",
        hovermode="x",
        xaxis_tickprefix="$",
        font=dict(family="Roboto"),
        plot_bgcolor="white",
        height=700,
        width=1000,
        annotations=steps[0]["args"][2]["annotations"],
    )

    fig.update_xaxes(
        visible=True,
        type="log",
        #  autorange=False,
        row=2,
        col=1,
    )
    fig.update_yaxes(
        dict(
            range=[0, 1],
            autorange=False,
            showspikes=False,  # Show spike line for y-axis
            # Format spike
            spikethickness=2,
            spikedash="dot",
            spikecolor="#999999",
            spikemode="toaxis",
        ),
        row=2,
        col=1,
    )

    return fig.show()


with warnings.catch_warnings():  # ignore "FutureWarning"
    """
    Ignore numpy 'FutureWarning' :
    "Using a non-tuple sequence for multidimensional indexing is deprecated;
    use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be
    interpreted as an array index, `arr[np.array(seq)]`, which will result
    either in an error or a different result."
    """
    warnings.filterwarnings("ignore", category=FutureWarning)
    ecdf_slider(simulated)


It is notable that while the KS D-statistics declines overall, it fluctuates until we get to $500 payments, at which point the D-statistic begins a steady decline to minimum of 0.300 under the $2,000 monthly benefit scenario. This is because the distances between the two curves are much larger around the center of the distributions than around the tails, thus the changes in the KS test are not picking up changes at lower incomes, where the curves were much closer together to begin with.

In our simulations, the net worth point at which the curves are furthest apart increases with the UBI amount. The net worth at which the gap is the biggest rises to just above $121,000, from $50,000 at baseline.


In [ ]:
fig = make_subplots(
    rows=2, 
    cols=1, 
    shared_xaxes=True,
    x_title='UBI per Month',
    subplot_titles=['KS Statistic', 'Most Unequal Net Worth']
    )


fig.add_trace(
    go.Scatter(
        x=sim_results["ubi_mo"],
        y=sim_results["ks_stat"],
        mode="lines+markers",
        name="ks_stat",
        marker_color=BLUE
    ),row=1,col=1
)
fig.add_trace(
    go.Scatter(
        x=sim_results["ubi_mo"],
        y=sim_results["most_unequal_networth_pa"],
        mode="lines+markers",
        name="Networth at KS D-stat",
         marker_color=LIGHT_BLUE,
         
    ),row=2, col=1
)

fig.update_layout(
    template=template,
    showlegend=False,
    hovermode="x",
    xaxis_tickprefix="$"
    )

common_layout_changes(fig)
add_ubi_center_logo(fig)

fig.show(config=CONFIG)


Further, without UBI the median White family has $6.45 for every $1 held by the median Black family. Under a $1000 monthly payment, this falls to $3.49 for every $1, and further down to $2.64 with a $2,000 monthly payment. 

The median net worth for White households rises from $115,103 without a UBI to $119,425 under a $1,000 monthly payment, and rises to $127,505 with a $2,000 monthly payment. For Black households, the median net worth rises from $17,853 to $34,183 with a $1,000 benefit, and further to $48,310 with a $2,000 benefit.

It should be noted that the median net worth per adult[<sup>1</sup>](#fn1) rises among White households, it merely rises by less than that of Black households.



<span id="fn1"> Household net worth conventionally adjust for number of adults, see example: [Credit Suisse: Global net worth Report](https://www.credit-suisse.com/about-us/en/reports-research/global-wealth-report.html)</span>

In [ ]:
fig = go.Figure()

hovertemplate= '$%{y:.0f}'

fig.add_trace(
    go.Scatter(
        x=sim_results["ubi_mo"],
        y=sim_results["Black_median_networth_pa"],
        mode="lines+markers",
        name="Black",
        marker_color=BLUE,
        hovertemplate=hovertemplate
        
    )
)
fig.add_trace(
    go.Scatter(
        x=sim_results["ubi_mo"],
        y=sim_results["White_median_networth_pa"],
        mode="lines+markers",
        name="White",
        marker_color=GRAY,
        hovertemplate=hovertemplate
    )
)


format_helper(
    fig,
    title="Median Net Worth per Adult by Race",
    xaxis_title="UBI per month",
    yaxis_title="Median net worth per adult",
    yaxis_ticksuffix="",
    yaxis_tickprefix="$",
    xaxis_tickprefix="$",
)

fig.show()


Unlike the rising median net worth for White households under the same UBI, mean net worth for White households declines from $557,216 without any UBI to $555,135 under a $1,000 monthly payment, and falls further to $553,054 with a $2000 monthly payment. For Black households, the mean net worth rises from $97,167 to $104,768 with a $1,000 benefit, and further to $112,369 with a $2,000 benefit.

In [ ]:
fig = go.Figure()

hovertemplate= '$%{y:.0f}'

fig.add_trace(
    go.Scatter(
        x=sim_results["ubi_mo"],
        y=sim_results["Black_mean_networth_pa"],
        mode="lines+markers",
        name="Black",
        marker_color=BLUE,
        hovertemplate=hovertemplate
        
    )
)
fig.add_trace(
    go.Scatter(
        x=sim_results["ubi_mo"],
        y=sim_results["White_mean_networth_pa"],
        mode="lines+markers",
        name="White",
        marker_color=GRAY,
        hovertemplate=hovertemplate
    )
)


format_helper(
    fig,
    title="Mean Net Worth per Adult by Race",
    xaxis_title="UBI per month",
    yaxis_title="Mean net worth per adult",
    yaxis_ticksuffix="",
    yaxis_tickprefix="$",
    xaxis_tickprefix="$",
)

fig.show()

Finally, we look at the share of households that fall below $50,000 per adult, the point at which those two curves were further apart in our baseline scenario. 

In [ ]:
fig = go.Figure()


fig.add_trace(
    go.Scatter(
        x=sim_results["ubi_mo"],
        y=sim_results['Black_share_above_50k']*100,
        mode="lines+markers",
        name="Black",
        marker_color=BLUE,
        hovertemplate=pct_str_format
    )
)
fig.add_trace(
    go.Scatter(
        x=sim_results["ubi_mo"],
        y=sim_results['White_share_above_50k']*100,
        mode="lines+markers",
        name="White",
        marker_color=GRAY,
        hovertemplate=pct_str_format
    )
)

format_helper(
    fig,
    title="Share of Households with Net Worth above $50k by Race",
    xaxis_title="UBI per month",
    yaxis_title="Share of households with net worth above $50k",
    yaxis_ticksuffix="%",
    yaxis_tickprefix="",
    xaxis_tickprefix="$",
)

fig.show()

The share of Black households with a per adult networth of over $50,000 rises from 65.9% to 61.2% when we raise the monthly UBI to $1,000 monthly, and falls further to 51.7% when we raise the benefit to $2,000 monthly.

In the scenarios we explore, the outcomes for Black households improve in the share of households with net worth per adult below $50,000 declines, median household net worth increases. In the final plot below, you can explore the relative changes in all of the metrics we’ve chosen.


# Create Plot With Sliders - `ecdf_slider`

In [ ]:
# create summary plot
fig=go.Figure()

def trace(col, chg_col, name, string, color):
  """
    Args:
        col: Column name.
        chg_col: Column name of change.
        name: Name of column for printing.
        string: Formatting of customdata[1].
        color: Line color.
    """
  fig.add_trace(
      go.Scatter(
          x=sim_results["ubi_mo"],
          y=sim_results[chg_col],
          mode="lines+markers",
          name=name,
          marker_color=color,
          marker=dict(size=5),
          customdata=np.stack((sim_results.ubi_mo, sim_results[col]), axis=-1),
            hovertemplate=
            # "<br>UBI/mo: %{x}<br>"
            # + "UBI: $%{customdata[0]: .0f}<br>"
            # + 
            name
            + ": "
            + string
            + "<br>"
            + "Percent change: %{y:.0f}%<br>",
          
      )
  )




trace(
    "White_mean_nw_as_pct_of_mean_Black",
    'White_mean_nw_as_pct_of_mean_Black_deltap',
    'White/Black mean net worth',
    "%{customdata[1]: .1f}%",
    DARK_BLUE)
# median net worth
trace(
    "White_median_nw_as_pct_of_median_Black",
    'White_median_nw_as_pct_of_median_Black_deltap',
    'White/Black median net worth',
    "%{customdata[1]: .1f}%",
    GRAY
    )
trace(
    "ks_stat",
    'ks_stat_deltap',
    'KS Statistic',
    "%{customdata[1]: .3f}",
    LIGHT_BLUE
    )
trace(
    "White_share_above_50k_pct_of_Black",
    'White_share_above_50k_pct_of_Black_deltap',
    'Share of Households over $50k: White/Black',
    "%{customdata[1]: .1f}%",
    BLUE
    )


format_helper(
    fig,
    title="How Different Measures of Racial Wealth Gap Change with UBI",
    xaxis_title="UBI per month",
    yaxis_title="Percent change in measures over baseline",
    yaxis_ticksuffix="%",
    yaxis_tickprefix="",
    xaxis_tickprefix="$",
)

# fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.75))

fig.show(config=CONFIG)

# fig.show()

However, we should note that this simulation does not show how these effects could grow over many years, nor does it account for differences in the composition of assets and debts each held by each household. [<sup>2</sup>](#fn2)

The power of a Universal Basic Income, however, goes beyond what we have shown in our model. The UBI approach does not rely on the discretion of employers - private or public - to conduct just hiring practices, the willingness of regulators to faithfully enforce anti-discrimination laws, nor the ability of individuals to navigate opaque and counterproductive eligibility criteria of safety net programs that have been long maligned and often underfunded. Above all else, the advantage of a UBI over other approaches is that once in place it will not rely on decades of good faith effort by those in power to keep it in place, but the support of all of those that will benefit. 





<span id="fn2"> Future researchers should explore alternative statistical tools that build off the KS test, such as the Cramer Von-Mises Test, which is more sensitive to outliers. 
</span>